In [ ]:
import numpy as np
from scipy.linalg import eig


T_matrix = np.array([[ 0.99980001,     0.01999867,  0.,          0.        ],
 [-0.01999867,  0.99980001,     0.,          0.,        ],
 [ 0.         , 0. ,         1. ,        -0.0031414  ],
 [ 0.          ,0.  ,        0.  ,        1.        ]] )

theta_Rho = np.array([-0.32955593,  2.05519271] )

theta_Rho_prime = np.array([-0.30955592,      2.05591822])


# [[ 0.99980001  0.01999867  0.          0.        ]
#  [-0.01999867  0.99980001  0.          0.        ]
#  [ 0.          0.          1.         -0.0031414 ]
#  [ 0.          0.          0.          1.        ]] [-0.32955593  2.05519271] [-0.30955592  2.05591822]

# Extract components from T_matrix
R_matrix = T_matrix[:3, :3]
t = T_matrix[:3, 3]

# Extract theta, theta_prime, R, R_prime from inputs
theta = theta_Rho[0]
theta_prime = theta_Rho_prime[0]
R = theta_Rho[1]
R_prime = theta_Rho_prime[1]

# Define linear constraints in matrix form A @ P = B
r1 = R_matrix[0, :]
r2 = R_matrix[1, :]

a1 = np.array([-1, np.tan(theta), 0, 0])
b1 = 0 
a2 = np.tan(theta_prime) * r2 - r1
a2 = np.hstack([a2, 0])
b2 = t[0] - np.tan(theta_prime) * t[1]
a3 = t.T @ R_matrix
a3 = np.hstack([a3, 0])
b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2
a4 = np.array([0, 0, 0, 1])
b4 = R**2

# 将线性方程组写成矩阵形式 A @ P = B
A = np.vstack([a1, a2, a3, a4])
b = np.array([b1, b2, b3, b4])

ATA_be = A.T @ A
ATb_be = A.T @ b
D = np.diag([1,1,1,0])
g = np.zeros(4)
g[3] = -0.5

# eig_lambda = eig(D, ATA_be, right=False)
eig_lambda = np.real(eig(D, ATA_be, right=False))

lambda_min = -1 / np.max(eig_lambda)


## 找二分法的初始左端点
lambda_l = lambda_min + 2

y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)
if y_l.T @ D @ y_l + 2 * g.T @ y_l < 0:    
    while True:
        y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)
        if y_l.T @ D @ y_l + 2 * g.T @ y_l > 0:
            break
        lambda_l -= 1
# already satisfy now we want to refine
while y_l.T @ D @ y_l + 2 * g.T @ y_l > 0:
    lambda_l += 0.1
    y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)
# 细调使其接近0
while y_l.T @ D @ y_l + 2 * g.T @ y_l <= 0:
    lambda_l -= 0.01
    y_l = np.linalg.solve(ATA_be + lambda_l * D, ATb_be - lambda_l * g)

# 找二分法的初始右端点
lambda_u = lambda_l + 0.1

# 二分法找最优lambda
times = 0
while (lambda_u - lambda_l) > 1e-12:
    times += 1
    lambda_temp = (lambda_u + lambda_l) / 2
    y_temp = np.linalg.solve(ATA_be + lambda_temp * D, ATb_be - lambda_temp * g)
    if (y_temp.T @ D @ y_temp + 2 * g.T @ y_temp) > 0:
        lambda_l = lambda_temp
    else:
        lambda_u = lambda_temp
    print(lambda_u, lambda_l,lambda_temp)
y = np.linalg.solve(ATA_be + lambda_u * D, ATb_be - lambda_u * g)
pos = y[:3]
print(pos)